# Data

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-15 01:19:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-03-15 01:19:22 (47.5 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [4]:
with open("input.txt",'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print(len(text))

1115394


In [6]:
print(text[0:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


# Tokenizer

## Simple_Character_LEvel

In [8]:
s_to_i = {ch:i for i, ch in enumerate(chars)}
i_to_s = {i:ch for i, ch in enumerate(chars)}


In [9]:
encode = lambda s: [s_to_i[ch] for ch in s]
decode = lambda l: ''.join([i_to_s[i] for i in l])

In [10]:
print(encode("Hello, there!"))

[20, 43, 50, 50, 53, 6, 1, 58, 46, 43, 56, 43, 2]


In [11]:
print(decode(encode("Hello, there!")))

Hello, there!


In [12]:
import torch
data = torch.tensor(encode(text), dtype= torch.long)
print(data.shape, data.dtype)
print(data[0:200])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


# Preparing for Training

In [13]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [14]:
block_size = 10
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When context is {context}, the target is : {target}")

When context is tensor([18]), the target is : 47
When context is tensor([18, 47]), the target is : 56
When context is tensor([18, 47, 56]), the target is : 57
When context is tensor([18, 47, 56, 57]), the target is : 58
When context is tensor([18, 47, 56, 57, 58]), the target is : 1
When context is tensor([18, 47, 56, 57, 58,  1]), the target is : 15
When context is tensor([18, 47, 56, 57, 58,  1, 15]), the target is : 47
When context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is : 58
When context is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]), the target is : 47
When context is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]), the target is : 64


In [16]:
torch.manual_seed(1337)
batch_size = 4
block_size = 10

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch(train_data)
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 10])
tensor([[53, 56,  1, 63, 53, 59,  1, 57, 46, 39],
        [ 1, 51, 43, 12,  0,  0, 28, 30, 27, 31],
        [39, 57,  1,  5, 58, 47, 57,  6,  0, 35],
        [ 1, 27,  1, 60, 47, 50, 43,  6,  0, 21]])
targets:
torch.Size([4, 10])
tensor([[56,  1, 63, 53, 59,  1, 57, 46, 39, 50],
        [51, 43, 12,  0,  0, 28, 30, 27, 31, 28],
        [57,  1,  5, 58, 47, 57,  6,  0, 35, 43],
        [27,  1, 60, 47, 50, 43,  6,  0, 21, 52]])


# Models

In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):


    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    

    def forward(self,idx, targets=None):
        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_token):
        #idx is a (B,T) array of indices in current context
        for _ in range(max_new_token):
            #get the prediction
            logits, loss = self(idx) #(B, T, C)
            #focus only in the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            #apply softmax to get probablities
            probs = F.softmax(logits, dim =-1) #(B,C)
            #sample for distributyion
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append sample imput to the runing sequence
            idx = torch.cat((idx, idx_next), dim =1) #(B, T+1)
        return idx
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)   
print(yb.shape)
print(loss)


torch.Size([40, 65])
torch.Size([4, 10])
tensor(4.7528, grad_fn=<NllLossBackward0>)


In [18]:
import tqdm as notebook_tqdm

In [19]:
#Create a Pytorch optimiser
optimiser = torch.optim.Adam(m.parameters(), lr = 1e-3)

/home/den/practice_linux/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
batch_size = 32
for steps in range(10000):

    #sample a batch of data
    xb, yb = get_batch("train")

    #evaluate the loss
    logits, loss = m(xb, yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()
print(loss.item())

2.500786542892456


In [21]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_token=500)[0].tolist()))


Ke sonidin y y a!
Caril.

AUCrenjohece w olld CHAL, mer thoun s's:Conchentilalllevise sthat dy hangilyoteng h hasbe pave pirance
RDe hicomyonthar's
PAS:
AKEd ith henouratucenonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercckehathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h haybet--s n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThineent.

Lavinde.
athave l.
KEONGBUCHandspo be y,-hedarwnoddy scar t tridesar, wne'shenou


#  Self Attention

## A mathematical trick

In [52]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
print(x[2,3])
x.shape

tensor([-1.1539, -0.2984])


torch.Size([4, 8, 2])

In [77]:
# Version !
# we want x[b,t] = mean_{i<-t} x[b,i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t, C)
        xbow[b,t] = torch.mean(xprev,0)

In [71]:
# version 2
wei = torch.tril(torch.ones(T,T))
wei = wei/ wei.sum(1, keepdim=True)
xbow2 = wei @ x    # (T, T) @ (B, T, C) batch matrix multlipication ------->(B, T, C)


In [76]:
# Version 3:
trill = torch.tril(torch.ones(T,T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(trill ==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True